In [3]:
#package loading 
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil.relativedelta import relativedelta
from scipy.stats import percentileofscore
from tqdm import tqdm
import cudf
import numba as nb
import os
import sys
import time
import pickle

import sklearn
import numba as nb
import functools as ft
import swifter

In [4]:
from utility import *
#del sys.modules["preprocess"]
from preprocess import * 

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [37]:
#define function for calculating ages
def calc_age(born_d,index_d):
   born_d = pd.to_datetime(born_d)
   index_d = pd.to_datetime(index_d)
   return index_d.year - born_d.year - ((index_d.month, 
                                      index_d.day) < (born_d.month, 
                                                    born_d.day))

In [38]:
pid_dx_subtypes = pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/pid_dx_cancer_subtypes.pkl','rb'))

#pasc and covid inex dates
index_date = pd.read_csv('/data/datasets/Tianchen/2022_covid_pasc_1FL_data/20221020/covid_pasc_cohort.csv')

In [123]:
data_wd = "/data/datasets/Tianchen/2022_covid_pasc_1FL_data/20221020/covid_database"

# covid database
diagnosis_cov = pd.read_csv(data_wd+"/DIAGNOSIS.csv",header=0,skiprows=[1])
procedure_cov = pd.read_csv(data_wd+"/PROCEDURES.csv",header=0,skiprows=[1])
demographics_cov = pd.read_csv(data_wd+"/DEMOGRAPHIC.csv",header=0,skiprows=[1])

data_wd = "/data/datasets/Tianchen/2022_covid_pasc_1FL_data/20221020/main_database"

# general database
diagnosis_m = pd.read_csv(data_wd+"/DIAGNOSIS.csv",header=0,skiprows=[1])
procedure_m = pd.read_csv(data_wd+"/PROCEDURES.csv",header=0,skiprows=[1])
demographics_m = pd.read_csv(data_wd+"/DEMOGRAPHIC.csv",header=0,skiprows=[1])

# union them
diagnosis = pd.concat([diagnosis_m,diagnosis_cov])
print("complete diagnosis")
procedure = pd.concat([procedure_m,procedure_cov])
print("complete procedure")
demographics = pd.concat([demographics_m,demographics_cov])
print("complete demographics")

#release memory
del(diagnosis_m)
del(diagnosis_cov)
del(procedure_m)
del(procedure_cov)
del(demographics_m)
del(demographics_cov)

complete diagnosis
complete procedure
complete demographics


# check the number of lung cancer patients that get the cancer after covid infection

## extract pid of lung cancers

In [124]:
pid_lung = list(set(pid_dx_subtypes.loc[pid_dx_subtypes['subtype']=="Lung"].reset_index(drop=True).ID))

## matching demographics and index dates

In [125]:
#choose those with covid19 index date
index_date = index_date.loc[~index_date['index_date'].isna()]

index_date['index_date'] = pd.to_datetime(index_date['index_date'])
index_date['pasc_onset_date'] = pd.to_datetime(index_date['pasc_onset_date'])

#index_date = index_date.loc[(index_date['index_date']<index_date['pasc_onset_date'])|index_date['pasc_onset_date'].isna()]

In [126]:
# This is because there are two database to be merged, i.e. covid database and general databse. The medical records are not duplicates abut the patient demographics are.
print(demographics.shape)
demographics = demographics.drop_duplicates(['ID'])
print(demographics.shape)

#pair demographics and index date
demographics = pd.merge(demographics,index_date,left_on='ID',right_on='ID',how='inner')
print(demographics.shape)

(455995, 12)
(229689, 12)
(225975, 15)


In [127]:
demographics = demographics[['ID',
                             'BIRTH_DATE',
                             'SEX',
                             'HISPANIC',
                             'RACE',
                             'PAT_PREF_LANGUAGE_SPOKEN',
                             'ZIP_CODE',
                             'index_date']]

In [128]:
demographics['index_age'] = demographics.swifter.apply(lambda x: calc_age(x.BIRTH_DATE, x.index_date), axis=1)

Pandas Apply:   0%|          | 0/225975 [00:00<?, ?it/s]

In [129]:
demographics

,ID,BIRTH_DATE,SEX,HISPANIC,RACE,PAT_PREF_LANGUAGE_SPOKEN,ZIP_CODE,index_date,index_age
0,19Oct2022covidpasc00121586,1991-10-04,F,N,05,ENG,327769298,2021-07-31,29
1,19Oct2022covidpasc00125493,1947-04-10,M,N,05,ENG,328188842,2020-07-26,73
2,19Oct2022covidpasc00008560,1961-08-09,M,Y,05,ENG,347412932,2022-01-06,60
3,19Oct2022covidpasc00183957,1975-10-15,F,N,05,ENG,327145836,2022-02-08,46
4,19Oct2022covidpasc00184103,1993-07-07,M,Y,05,NaN,33837,2021-01-07,27
...,...,...,...,...,...,...,...,...,...
225970,19Oct2022covidpasc00223324,2021-01-28,M,OT,05,ENG,NaN,2022-06-04,1
225971,19Oct2022covidpasc00001449,2000-01-18,M,OT,OT,SPA,NaN,2022-06-04,22
225972,19Oct2022covidpasc00093000,1954-12-11,M,UN,03,NaN,32351,2022-01-28,67
225973,19Oct2022covidpasc00205743,1987-12-01,F,N,05,NaN,32312,2022-01-13,34


## get the 1st diagnosis date

In [130]:
dx_codes_icd9_Lungcaner = tuple(['1622', '1623', '1624', '1625', '1628', '1629', '2312', 'V1011'])
dx_codes_icd10_Lungcaner = tuple(['C34'])

In [131]:
#diagnosis
pid_dx_lung_diag = extract_cancer_pid_time(df=diagnosis,\
                                     extract_from_col="DX",\
                                     code_type_col="DX_TYPE",\
                                     date_col = "DX_DATE",\
                                     codes_icd9=dx_codes_icd9_Lungcaner,\
                                     codes_icd10=dx_codes_icd10_Lungcaner,\
                                     subtype="lung")

#procedure
pid_dx_lung_procedure = extract_cancer_pid_time(df=procedure,\
                                          extract_from_col="PX",\
                                          code_type_col="PX_TYPE",\
                                          date_col = "PX_DATE",\
                                          codes_icd9=dx_codes_icd9_Lungcaner,\
                                          codes_icd10=dx_codes_icd10_Lungcaner,\
                                          subtype="lung")

In [132]:
pid_dx_lung = pd.concat([pid_dx_lung_diag,pid_dx_lung_procedure]).\
                drop_duplicates(ignore_index=True)

In [133]:
pid_dx_lung['DX_DATE'] = pd.to_datetime(pid_dx_lung['DX_DATE'])
pid_dx_lung['ADMIT_DATE'] = pd.to_datetime(pid_dx_lung['ADMIT_DATE'])

In [134]:
dx_lung_date = pid_dx_lung.groupby('ID').agg({'DX_DATE':'min',
                               'ADMIT_DATE':'min'}).reset_index()

In [135]:
dx_lung_date['INDEX_DATE_LUNG'] = dx_lung_date[['DX_DATE','ADMIT_DATE']].apply('min',axis=1)

In [136]:
print(dx_lung_date.shape)
dx_lung_date = dx_lung_date.merge(demographics,on='ID',how='inner')
print(dx_lung_date.shape)

(2024, 4)
(1989, 12)


In [137]:
print(dx_lung_date.shape)

(1989, 12)


## get those with lung cancer after covid

In [138]:
dx_lung_afterCOVID = dx_lung_date.loc[dx_lung_date['INDEX_DATE_LUNG'] > dx_lung_date['index_date']].reset_index(drop=True)

In [139]:
dx_lung_afterCOVID['RACE']=dx_lung_afterCOVID['RACE'].replace({'01':'American Indian or Alaska Native',
                            '02':'Asian',
                            '03':'Black or African American',
                            '04':'Native Hawaiian or Other Pacific Islander',
                            '05':'White',
                            '06':'Multiple race',
                            '07':'Refuse to answer',
                            'NI':'No information',
                            'UN':'Unknown',
                            'OT':'Other' })
dx_lung_afterCOVID['RACE'] = dx_lung_afterCOVID['RACE'].fillna('No information')
dx_lung_afterCOVID['RACE_Ethinic'] = np.where( (dx_lung_afterCOVID['HISPANIC']=='N') & (dx_lung_afterCOVID['RACE']=='White'),
                                              'NHW',
                                              np.where((dx_lung_afterCOVID['HISPANIC']=='N') & (dx_lung_afterCOVID['RACE']=='Black or African American'),
                                                      'NHB',
                                                      np.where(dx_lung_afterCOVID['HISPANIC']=='Y',
                                                               'Hispanics',
                                                              np.where(dx_lung_afterCOVID['RACE'].isin(['Unknown','No information']),
                                                                       'Unknown',
                                                                       'Other')
                                                      )
                                             ))

#calculate age at diagnosis (index_date provided)
dx_lung_afterCOVID['index_age'] = dx_lung_afterCOVID.swifter.apply(lambda x: calc_age(x.BIRTH_DATE, x.index_date), axis=1)
#calc group
dx_lung_afterCOVID['age_band'] = pd.cut(dx_lung_afterCOVID.index_age, \
                                    [-999,18,65,75,999],\
                                    labels = ['[0,18)','[18,64)','[65,75)','>=75'],\
                                    include_lowest=True,right=False)

Pandas Apply:   0%|          | 0/256 [00:00<?, ?it/s]

In [142]:
dx_lung_afterCOVID['age_band2'] = pd.cut(dx_lung_afterCOVID.index_age, \
                                    [-999,20,30,40,45,50,60,70,80,999],\
                                    labels = ['[0,20)','[20,30)','[30,40)','[40,45)','[45,50)','[50,60)','[60,70)','[70,80)','80+'],\
                                    include_lowest=True,right=False)

In [158]:
dx_lung_afterCOVID

,ID,DX_DATE,ADMIT_DATE,INDEX_DATE_LUNG,BIRTH_DATE,SEX,HISPANIC,RACE,PAT_PREF_LANGUAGE_SPOKEN,ZIP_CODE,index_date,index_age,RACE_Ethinic,age_band,age_band2
0,19Oct2022covidpasc00000551,NaT,2021-09-01,2021-09-01,2001-03-30,F,N,Black or African American,NaN,334626193,2020-06-27,19,NHB,"[18,64)","[0,20)"
1,19Oct2022covidpasc00000928,NaT,2022-03-21,2022-03-21,1979-05-29,F,N,Black or African American,NaN,333083716,2021-08-29,42,NHB,"[18,64)","[40,45)"
2,19Oct2022covidpasc00001322,NaT,2021-01-26,2021-01-26,1940-03-08,M,N,White,ENG,327125663,2021-01-20,80,NHW,>=75,80+
3,19Oct2022covidpasc00002280,2020-08-05,2020-08-05,2020-08-05,1949-06-04,M,Y,White,SPA,331281125,2020-07-20,71,Hispanics,"[65,75)","[70,80)"
4,19Oct2022covidpasc00003203,2021-05-08,2021-05-08,2021-05-08,1935-08-09,F,OT,Asian,NaN,32832,2020-12-10,85,Other,>=75,80+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,19Oct2022covidpasc00226500,NaT,2020-10-29,2020-10-29,1947-01-29,F,UN,Unknown,NI,34761,2020-08-14,73,Unknown,"[65,75)","[70,80)"
252,19Oct2022covidpasc00227292,2021-01-15,2021-01-15,2021-01-15,1949-01-01,F,N,Black or African American,ENG,33162,2020-10-04,71,NHB,"[65,75)","[70,80)"
253,19Oct2022covidpasc00228339,NaT,2021-12-03,2021-12-03,1934-12-03,M,N,White,ENG,327891473,2020-12-30,86,NHW,>=75,80+
254,19Oct2022covidpasc00228821,NaT,2021-12-09,2021-12-09,2001-12-13,M,N,Black or African American,NaN,319034125,2021-09-20,19,NHB,"[18,64)","[0,20)"


# CCI=0

In [85]:
#output to r
# covid database
data_wd = "/data/datasets/Tianchen/2022_covid_pasc_1FL_data/20221020/covid_database"
diagnosis_cov = pd.read_csv(data_wd+"/DIAGNOSIS.csv",header=0,skiprows=[1])
procedure_cov = pd.read_csv(data_wd+"/PROCEDURES.csv",header=0,skiprows=[1])

data_wd = "/data/datasets/Tianchen/2022_covid_pasc_1FL_data/20221020/main_database"
# general database
diagnosis_m = pd.read_csv(data_wd+"/DIAGNOSIS.csv",header=0,skiprows=[1])
procedure_m = pd.read_csv(data_wd+"/PROCEDURES.csv",header=0,skiprows=[1])

# union them
diagnosis = pd.concat([diagnosis_m,diagnosis_cov])
print("complete diagnosis")
procedure = pd.concat([procedure_m,procedure_cov])
print("complete procedure")

#release memory
del(diagnosis_m)
del(diagnosis_cov)
del(procedure_m)
del(procedure_cov)

complete diagnosis
complete procedure


In [87]:
diagnosis_uniq = diagnosis[['ID','DX','DX_TYPE']].drop_duplicates(ignore_index=True)
procedure_uniq = procedure[['ID','PX','PX_TYPE']].drop_duplicates(ignore_index=True)

In [88]:
diagnosis_uniq.to_csv('/data/datasets/changyuyin/2_COVIDwithCancer/diagnosis_uniq.csv',index=False)
procedure_uniq.to_csv('/data/datasets/changyuyin/2_COVIDwithCancer/procedure_uniq.csv',index=False)

In [89]:
diagnosis_uniq.shape,procedure_uniq.shape

((19861106, 3), (18765380, 3))

In [1]:
from comorbidipy import comorbidity

In [6]:
diagnosis_uniq = pd.read_csv('/data/datasets/changyuyin/2_COVIDwithCancer/diagnosis_uniq.csv')

In [7]:
procedure_uniq = pd.read_csv('/data/datasets/changyuyin/2_COVIDwithCancer/procedure_uniq.csv')

In [15]:
procedure_uniq = procedure_uniq[procedure_uniq['PX_TYPE'].isin(['09','10',10,9,'9'])]
procedure_uniq.columns = ['ID','DX','DX_TYPE']

diagnosis_uniq = diagnosis_uniq[diagnosis_uniq['DX_TYPE'].isin(['09','10',10,9,'9'])]

In [49]:
df = pd.concat([diagnosis_uniq,procedure_uniq]).drop_duplicates(ignore_index=True)

In [50]:
df['DX'] = df['DX'].str.replace('[^a-zA-Z0-9]', '')

In [52]:
df = df.merge(demographics[['ID','index_age']],how='left',on='ID')

## calculate scores

In [103]:
charlson9 = comorbidity(df, 
                         "ID", 
                         code = "DX", 
                         age=None,
                         icd='icd9',
                         score = "charlson", 
                         variant="quan",
                         weighting="quan",
                         assign0 = False)

KeyboardInterrupt: 

In [74]:
charlson10 = comorbidity(df, 
                         "ID", 
                         code = "DX", 
                         age=None,
                         icd='icd10',
                         score = "charlson", 
                         variant="quan",
                         weighting="quan",
                         assign0 = False)

In [80]:
charlson_all = pd.concat([charlson9,charlson10]).groupby('ID').sum().reset_index()

In [91]:
charlson_all = demographics[['ID']].merge(charlson_all,how='left',on='ID').fillna(0)

# Non-cancer covid patient

In [150]:
all_cancer_pid = pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/pid_dx_cancer_subtypes.pkl','rb'))

In [151]:
pt_noncancer = demographics[~demographics['ID'].isin(all_cancer_pid.ID)]

In [152]:
pt_noncancer['RACE']=pt_noncancer['RACE'].replace({'01':'American Indian or Alaska Native',
                            '02':'Asian',
                            '03':'Black or African American',
                            '04':'Native Hawaiian or Other Pacific Islander',
                            '05':'White',
                            '06':'Multiple race',
                            '07':'Refuse to answer',
                            'NI':'No information',
                            'UN':'Unknown',
                            'OT':'Other' })
pt_noncancer['RACE'] = pt_noncancer['RACE'].fillna('No information')
pt_noncancer['RACE_Ethinic'] = np.where( (pt_noncancer['HISPANIC']=='N') & (pt_noncancer['RACE']=='White'),
                                              'NHW',
                                              np.where((pt_noncancer['HISPANIC']=='N') & (pt_noncancer['RACE']=='Black or African American'),
                                                      'NHB',
                                                      np.where(pt_noncancer['HISPANIC']=='Y',
                                                               'Hispanics',
                                                              np.where(pt_noncancer['RACE'].isin(['Unknown','No information']),
                                                                       'Unknown',
                                                                       'Other')
                                                      )
                                             ))

#calculate age at diagnosis (index_date provided)
pt_noncancer['index_age'] = pt_noncancer.swifter.apply(lambda x: calc_age(x.BIRTH_DATE, x.index_date), axis=1)
#calc group
pt_noncancer['age_band'] = pd.cut(pt_noncancer.index_age, \
                                    [-999,18,65,75,999],\
                                    labels = ['[0,18)','[18,64)','[65,75)','>=75'],\
                                    include_lowest=True,right=False)

Pandas Apply:   0%|          | 0/194770 [00:00<?, ?it/s]

In [153]:
pt_noncancer['age_band2'] = pd.cut(pt_noncancer.index_age, \
                                     [-999,20,30,40,45,50,60,70,80,999],\
                                    labels = ['[0,20)','[20,30)','[30,40)','[40,45)','[45,50)','[50,60)','[60,70)','[70,80)','80+'],\
                                    include_lowest=True,right=False)

In [154]:
pt_noncancer['age_band2'].value_counts()

[0,20)     38998
[20,30)    37854
[30,40)    31787
[50,60)    22582
[60,70)    16590
[40,45)    14102
[45,50)    12609
[70,80)    11228
80+         9020
Name: age_band2, dtype: int64

In [155]:
pt_noncancer = pt_noncancer.merge(charlson_all[['ID','comorbidity_score']],how='left',on='ID')

In [156]:
pt_noncancer_healthy = pt_noncancer[pt_noncancer['comorbidity_score']==0]

In [157]:
pt_noncancer_healthy['age_band2'].value_counts()

[0,20)     24522
[20,30)    24237
[30,40)    19420
[50,60)    12262
[40,45)     8322
[60,70)     7293
[45,50)     7230
[70,80)     4077
80+         1961
Name: age_band2, dtype: int64

In [162]:
dx_lung_afterCOVID['age_band2'].value_counts()

[60,70)    65
[70,80)    46
[50,60)    28
80+        28
[20,30)    21
[30,40)    20
[0,20)     18
[45,50)    16
[40,45)    14
Name: age_band2, dtype: int64

In [161]:
len(set(dx_lung_afterCOVID.ID))

256

# summary

In [69]:
def demog_summary(demographics):
    demogrpahics_tmp = demographics[['ID','SEX','RACE_Ethinic','age_band']].drop_duplicates(ignore_index=True)
    print(demographics.SEX.value_counts())
    print(demographics.RACE_Ethinic.value_counts())
    print(demographics.age_band.value_counts())

In [70]:
demog_summary(pt_noncancer)

F     112275
M      82454
NI        20
UN        12
OT         9
Name: SEX, dtype: int64
Hispanics    70501
NHW          57594
NHB          46491
Other        13507
Unknown       6677
Name: RACE_Ethinic, dtype: int64
[18,64)    139996
[0,18)      27239
>=75        13868
[65,75)     13667
Name: age_band, dtype: int64


In [71]:
demog_summary(dx_lung_afterCOVID)

F    152
M    104
Name: SEX, dtype: int64
NHW          102
Hispanics     70
NHB           62
Other         16
Unknown        6
Name: RACE_Ethinic, dtype: int64
[18,64)    140
[65,75)     56
>=75        51
[0,18)       9
Name: age_band, dtype: int64
